In [1]:
import numpy as np
import pandas as pd

In [2]:
slide_mes = 0
slide_noise = 0
mem_amount = 5

In [3]:
u_future = np.random.normal(0,2,60)
u = u_future[0 + slide_noise  : (slide_noise + mem_amount)]


In [4]:
c = np.random.normal(0,10,mem_amount)
h = np.ones(mem_amount)
n_moment = np.random.normal(0,1,1)

E = np.zeros(15)
X = np.zeros(15)
Y = np.zeros(15)

In [5]:
def slide_measurements():
    global slide_mes
    slide_mes = slide_mes + 1
    u = u_future[0 +slide_mes  : (slide_mes + mem_amount)]

In [6]:
full_measurments = np.arange(50)

In [7]:
x = np.add(np.dot(u,h),float(n_moment))

y = np.dot(h,x)
e = y -x 

E = np.append(E, e)
X = np.append(X,x)
Y = np.append(Y,y)
slide_measurements()

In [8]:
#Initate parameters and variables for test 
mem_size = 3
time = 0
measure = np.array(full_measurments[time:mem_size]) #Recent recorded measurments -- array size is same as memory size
channels = np.array(np.arange(mem_size) +1)
h = np.ones(mem_size) # Estimation vector
X_test = np.zeros(mem_size)
Y_test = np.zeros(mem_size)
E_test = np.zeros(mem_size)


In [9]:
P  = np.zeros(50)

forgetting_factor = 0.3
q_test = np.zeros(mem_size)
p_test = np.zeros(mem_size)                #Dimension is (1,mem)
R_x_matrix = np.zeros((mem_size,mem_size)) #Dimension is (mem,mem)
forgetting_factor_matrix =  np.array([(1 ,0, 0),(0, forgetting_factor, 0),(0, 0, forgetting_factor**2)])

In [10]:
# Update the measurments
def update_measurments():
    global time ,measure
    time = time + 1
    measure = np.array(full_measurments[time:time+3])

In [11]:
## >> Initiating signals
#Determine x and append to X

x = np.convolve(channels,measure)
assert(len(x) == len(measure) + len(channels)-1)
# The above not neccesery for debug
x  = np.convolve(channels,measure,"valid")
assert(len(x) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
x = np.add(x,n_moment)

#q_test array is filled as FILO
q_test= np.insert(q_test,0,x)
q_test= np.delete(q_test,(len(q_test)-1))

In [12]:
#Determine y and e append to Y

y = np.convolve(h,X_test,"valid")
assert(len(y) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".

#X_test array is filled as FILO
Y_test= np.insert(Y_test,0,y)
Y_test= np.delete(Y_test,(len(Y_test)-1))

In [13]:
# Determine the error

# Although the first estimation can be ready after some delay
# the estimations are in real time 
# thus error can be determined as

e = np.subtract(measure[2],y)
assert(len(e) == 1)

#X_test array is filled as FILO
E_test= np.insert(E_test,0,e)
E_test= np.delete(E_test,(len(E_test)-1))

In [14]:
#Loading Q array for A_t representation
h = np.zeros(mem_size)      # Solution vector initation
P = np.zeros((mem_size,mem_size))

In [15]:
q_test

array([4.90501044, 0.        , 0.        ])

In [ ]:
measure[i]
time

In [ ]:
#Determining p array recursively

def update_p():
    global time,p_test,measure,q_test
    assert(np.shape(p_test) == (3,)) # p_test is not correct shape
    if (time > mem_size):
        p_test = forgetting_factor*p_test + q_test*measure[2]
    elif(time == mem_size):
        # First meanigful p is determined after num of recorded measurements are equal to num of memory
        for k in range (3):
            p_test = np.zeros(mem_size)
            p_test =  p_test + np.transpose(q_test)*measure[k]*forgetting_factor**k

In [ ]:
#Update the new p 
update_p()

#Store the p for debug purposes
P_store = np.append(P,p_test)

In [ ]:
#Function for determining R_x matrix recursively

def update_R_x():
    global time,measure,R_x_matrix,q_test
    if (time > mem_size):
        R_x_matrix = forgetting_factor*R_x_matrix + np.outer(np.transpose(q_test), q_test)
    elif(time == mem_size):
        # First meanigful R_x is determined after num of recorded measurements are equal to num of memory
        R_x_matrix =  np.outer(np.transpose(q_test), q_test) * forgetting_factor_matrix

In [ ]:
# Update R_x matrix recursively
update_R_x()

In [ ]:
# Althogh what we need is inverse of R_x and not R_x
# Thankfully there is easier method to update inverse of R_x

def update_P():
    global time,measure,R_x_matrix,q_test,P,k
    
    if (time > mem_size):    
        #Determing  Kalman Gain Factor
        k = np.dot(P,q_test)/(forgetting_factor+ np.dot(np.dot(np.transpose(q_test),P),np.transpose(q_test)))
        # Sherman-Morrison formula for updating the inverse of R_x
        P = (forgetting_factor**-1)*P -((forgetting_factor**-1)* np.dot(k,np.transpose(q_test))*P)
    elif(time == mem_size):
        # First meanigful P is determined after num of recorded measurements are equal to num of memory
        assert(np.linalg.det(R_x_matrix) == 0) 
        # Zero determinant matrix inversion not possible wait for another measurement
        P = np.linalg.inv(R_x_matrix)

In [ ]:
#Update P (Inverse of R_x)
update_P()

In [ ]:
def update_solution():
    global time,h,k,measure

    if (time > mem_size):
        h = h + k*(measure -np.transpose(q_test)*h) 
    elif(time == mem_size):
        pass

In [ ]:
# Update the solution
update_solution()

In [19]:
class rls_algo:
    def __init__(self,total_measurement,memory_size,forgetting_factor):
        self.total_measurement = total_measurement
        self.memory_size = memory_size
        self.time = 0
        self.measure = np.zeros(memory_size)
        assert(forgetting_factor < 1) # forgetting factor should be 0<x<1
        self.forgetting_factor = forgetting_factor
        self.forgetting_factor_matrix = np.array([(1 ,0, 0),(0, forgetting_factor, 0),(0, 0, forgetting_factor**2)])

        self.channels = np.linspace(1,0.2,memory_size) #Fading channel
        self.h =  np.ones(mem_size)                    # Estimation vector

        # Initiate neccesery matrices
        self.q_test     = np.zeros(mem_size)            #Dimension is (,mem)
        self.p_Test     = np.zeros(mem_size)            #Dimension is (,mem)
        self_Rx_martrix = np.zeros((mem_size,mem_size)) #Dimension is (mem,mem)
        self_P          = np.zeros((mem_size,mem_size)) #Dimension is (mem,mem)
        self.k          = np.zeros(mem_size)            #Dimension is (,mem)

        self.x = 0
        self.y = 0

    # Update the measurments
    #####################################################
    def update_measurments(self):
        self.time = self.time + 1
        self.measure = np.array(self.total_measurement[self.time:self.time+3])

    #Generate x with new measurements
    #####################################################
    def generate_x(self):
        self.x  = np.convolve(self.channels,self.measure,"valid")
        assert(len(self.x) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
        n_moment = np.random.normal(0,1,1)
        self.x = np.add(self.x,n_moment)

        #q_test array is filled as FILO
        self.q_test = np.insert( self.q_test ,0,self.x)
        self.q_test = np.delete( self.q_test ,(len(self.q_test)-1))

    #Generate y with x and previous estimates
    #####################################################
    def generate_y(self):
        #Determine y and e append to Y
        self.y = np.convolve(h,self.q_test,"valid")
        assert(len(y) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
    
    #Determine p array
    #####################################################
    def update_p(self):
        assert(np.shape(self.p_test) == (3,)) # p_test is not correct shape
        if (self.time > self.mem_size):
            self.p_test = self.forgetting_factor*self.p_test + self.q_test*self.measure[2]
        elif(self.time == self.mem_size):
            # First meanigful p is determined after num of recorded measurements are equal to num of memory
            for k in range (self.mem_size):
                self.p_test =  np.zeros(self.mem_size)
                self.p_test =  self.p_test + np.transpose(self.q_test)*self.measure[k]*self.forgetting_factor**self.k

    #Function for determining R_x matrix recursively
    #####################################################
    def update_R_x(self):
        if (self.time > self.mem_size):
            self.R_x_matrix = self.forgetting_factor*self.R_x_matrix + np.outer(np.transpose(self.q_test), self.q_test)
        elif(self.time == self.mem_size):
            # First meanigful R_x is determined after num of recorded measurements are equal to num of memory
            self.R_x_matrix =  np.outer(np.transpose(self.q_test), self.q_test) * self.forgetting_factor_matrix

    #Function for updating { P  == inverse of R_x } recursively
    #####################################################
    def update_P(self):
        if (self.time > self.mem_size):    
            #Determing  Kalman Gain Factor
            self.k = np.dot(self.P,self.q_test)/(self.forgetting_factor+ np.dot(np.dot(np.transpose(self.q_test),self.P),np.transpose(self.q_test)))
            # Sherman-Morrison formula for updating the inverse of R_x
            self.P = (self.forgetting_factor**-1)*self.P -((self.forgetting_factor**-1)* np.dot(self.k,np.transpose(self.q_test))*self.P)
        elif(self.time == self.mem_size):
            # First meanigful P is determined after num of recorded measurements are equal to num of memory
            assert(np.linalg.det(self.R_x_matrix) == 0) 
            # Zero determinant matrix inversion not possible wait for another measurement
            self.P = np.linalg.inv(self.R_x_matrix)

    #Function for updating {-- The Estimate --} recursively
    #####################################################
    def update_solution(self):
        if (self.time > self.mem_size):
            self.h = self.h + self.k*(self.measure -np.transpose(self.q_test)*self.h) 
        elif(self.time == self.mem_size):
            pass


    #  Reset variables
    #####################################################
    def reset_timer(self):
        self.time    = 0
        self.measure = np.zeros(self.memory_size)
        self.x = 0
        self.y = 0
        self.q_test     = np.zeros(mem_size)            
        self.p_Test     = np.zeros(mem_size)     
        self_Rx_martrix = np.zeros((mem_size,mem_size)) #Dimension is (mem,mem)
        self.k          = np.zeros(mem_size)            #Dimension is (,mem)       
        self_P          = np.zeros((mem_size,mem_size)) #Dimension is (mem,mem)

        

In [20]:
My_algo = rls_algo(total_measurement = np.arange(20), memory_size=3,forgetting_factor=0.03 )

In [21]:
My_algo.reset_timer()

for i in range(5):
    print("New noisy input",My_algo.x)
    print("Noisy Measurments in memory", My_algo.q_test)
    print("Uncorropted measurments in memory-> ", My_algo.measure)
    print("New y-> ", My_algo.y)
    My_algo.update_measurments()
    My_algo.generate_x()
    My_algo.generate_y()

New noisy input 0
Noisy Measurments in memory [0. 0. 0.]
Uncorropted measurments in memory->  [0. 0. 0.]
New y->  0
New noisy input [5.11652175]
Noisy Measurments in memory [5.11652175 0.         0.        ]
Uncorropted measurments in memory->  [1 2 3]
New y->  [0.]
New noisy input [6.04147139]
Noisy Measurments in memory [6.04147139 5.11652175 0.        ]
Uncorropted measurments in memory->  [2 3 4]
New y->  [0.]
New noisy input [7.79603597]
Noisy Measurments in memory [7.79603597 6.04147139 5.11652175]
Uncorropted measurments in memory->  [3 4 5]
New y->  [0.]
New noisy input [10.88568377]
Noisy Measurments in memory [10.88568377  7.79603597  6.04147139]
Uncorropted measurments in memory->  [4 5 6]
New y->  [0.]


In [22]:
My_algo.generate_y()

In [ ]:
My_algo.y

array([0.])